In [3]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
# import cartopy.crs as ccrs
#set directory for importinh my_tools,py
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from datetime import datetime
import glob
%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#set path. Change accoridngly
path_file = '/media/onno/Algemeen/Thesis/'
path = '/media/onno/Algemeen/Thesis/GFS_mean_season/'

Mean variable error for each season

In [2]:
#Categorize each month for season
variablez = ['envelope','phasespeed','u_wind']
seasons = {'JJA':[6,7,8],
          'DJF':[12,1,2]}
#Loop through season
for variable in variablez:
    print(variable)
    #get file names from imported dictionary
    file_fcst = file_dic[variable]['ERA5RF']
    file_rean = file_dic[variable]['ERA5']
    #open netcdf files as xarray datasets
    fcst = xr.open_dataset(path_file+file_fcst,decode_times=False).squeeze()
    rean = xr.open_dataset(path_file+file_rean,decode_times=False).squeeze()
    #change dimension names of ERA5 dataset
    if list(rean.coords)[1]=='longitude':
        rean=rean.rename({'longitude':'lon'})
    if list(rean.coords)[2]=='latitude':
        rean=rean.rename({'latitude':'lat'})

    #Convert dates to standard datetime values
    fcst['time']=my_tools.convert_date_era5rf_r(fcst.time.values)
    rean['time']=my_tools.convert_date_era_r(rean.time.values)

    #Select only data for 24h intervals
    fcst = fcst.sel(lead=fcst.lead.values[[bool(i) for i in fcst.lead%24==0]],
                   time=fcst.time.values[[bool(i) for i in fcst.time.dt.hour%24==0]])  
    rean = rean.sel(time=rean.time.values[[bool(i) for i in rean.time.dt.hour%24==0]])
    for key in seasons:
        #loop through lead days
        for i in range(0,11):
            if (variable=='phasespeed')&(i>=6):
                continue
            #Select days for which GEFS data is present and make datasets equally long
            fcst_d = fcst.sel(time=slice('1984-12-1','2019-11-{}'.format(30-i)),lead=i*24)
            rean_d = rean.sel(time=slice('1984-12-{}'.format(1+i),'2019-11-30'))        
            #Make boolean array for each day that is present in season        
            idxz = np.isin(fcst_d.time.dt.month,seasons[key])
            fcst_season = fcst_d[plot_dic[variable]['data_variable']][idxz] ; rean_season = rean_d[plot_dic[variable]['data_variable']][idxz]
            #Check if dates are correct
            assert(np.all(np.isin([(j-pd.Timedelta(i,'d')).month for j in rean_season.time.values],seasons[key])))
            assert(np.all(np.isin([pd.to_datetime(j).month for j in fcst_season.time.values],seasons[key])))
            fcst_season_mean = fcst_season.mean(dim='time')
            rean_season_mean = rean_season.mean(dim='time')
            diff = fcst_season_mean - rean_season_mean
            diff.to_netcdf(path+'/ERA5RF_mean_error_{}_{}_day_{:02d}.nc'.format(plot_dic[variable]['filename'],key,i))



u_wind


NameError: name 'path_file' is not defined